In [ ]:
from train_utils import DrivingData
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook
import ipywidgets
import numpy as np
import bokeh.plotting as bkp
from bokeh.io import output_notebook, push_notebook
# 读取数据
path = ["/cailiu2/Diffusion-Planner/data/processed/us-nv-las-vegas-strip_e64faffc07295e8f.npz"]
output_notebook()

train_set = DrivingData(path, 10, 10)
ego, agents, map_lanes, map_crosswalks, route_lanes, ego_future_gt, agents_future_gt, _, _ = train_set[0]

"""
<ego>: <ego> (including x, y, theta), the anchor state is defined as ego[-1, :]
     <ego> include anchor state past 20 frames state info, and all history states are converted to relative coords of anchor state

<agents>: [num_agents, num_past_frames, agent_state], agent_state include (x, y, heading) velocities (vx, vy, yaw rate) and size (length, width)
<mao>
"""

# 确保数据维度正确
print("ego shape:", ego.shape)  # 应为 (num_frames, 2)
print("agents shape:", agents.shape)  # 应为 (num_agents, num_frames, 2)

num_frames = len(ego)
num_agents = len(agents)

# 提取ego和agents的信息
ego_x, ego_y = ego[:, 0], ego[:, 1]
agents_x, agents_y = agents[:, :, 0], agents[:, :, 1]

ancor_state = ego[-1, :]

# 提取map lanes信息
num_map_lanes = len(map_lanes)
num_points_of_lane = len(map_lanes[0])

lane_xs=[map_lanes[i, :, 0].tolist() for i in range(num_map_lanes)]
lane_ys=[map_lanes[i, :, 1].tolist() for i in range(num_map_lanes)]

#提取 map_crosswalks信息
num_crosswalks = len(map_crosswalks)
num_points_of_crosswalks = len(map_crosswalks[0])
crosswalks_xs = [map_crosswalks[i, :, 0].tolist() for i in range(num_crosswalks)]
crosswalks_ys = [map_crosswalks[i, :, 1].tolist() for i in range(num_crosswalks)]

#提取route lanes信息
num_route_lane = len(route_lanes)
num_points_of_route_lane = len(route_lanes[0])
route_lane_xs=[route_lanes[i, :, 0].tolist() for i in range(num_route_lane)]
route_lane_ys=[route_lanes[i, :, 1].tolist() for i in range(num_route_lane)]

#提取未来8s的自车真实轨迹
ego_future_gt_xs = ego_future_gt[:, 0].tolist()
ego_future_gt_ys = ego_future_gt[:, 1].tolist()

#提取未来8s障碍车的真实轨迹
num_agents_future_gt = agents_future_gt.shape[0]
agents_future_gt_xs = [agents_future_gt[i, :, 0].tolist() for i in range(num_agents_future_gt)]
agents_future_gt_ys = [agents_future_gt[i, :, 1].tolist() for i in range(num_agents_future_gt)] #note the agents coord is relative


# 初始视图范围
initial_x, initial_y = ego_x[0], ego_y[0]
p = bkp.figure(
    title="Ego Vehicle & agents",
    x_range=(initial_x - 50, initial_x + 50),
    y_range=(initial_y - 50, initial_y + 50),
    width=600, height=600
)

# 数据源
source = ColumnDataSource(data={
    'ego_x': [],
    'ego_y': [],
})

agents_source = ColumnDataSource(data={
    'agents_x': agents_x[:, 0].tolist(),
    'agents_y': agents_y[:, 0].tolist()
})

map_lane_source = ColumnDataSource(data={'lane_xs': lane_xs, 'lane_ys': lane_ys})

map_crosswalks_source = ColumnDataSource(data={'crosswalks_xs': crosswalks_xs, 'crosswalks_ys': crosswalks_ys})

route_lane_source = ColumnDataSource(data={'lane_xs': route_lane_xs, 'lane_ys': route_lane_ys})

ego_future_gt_source = ColumnDataSource(data={'xs': ego_future_gt_xs, 'ys': ego_future_gt_ys})

agents_future_gt_source = ColumnDataSource(data={
    'agents_x': agents_future_gt_xs,
    'agents_y': agents_future_gt_ys
})
# 绘图
p.circle('ego_x', 'ego_y', source=source, size=12, color="red", legend_label="Ego Vehicle")
p.scatter('agents_x', 'agents_y', source=agents_source, size=6, color="blue", alpha=0.6, legend_label="agents")

p.multi_line('lane_xs', 'lane_ys', source=map_lane_source, line_width=2, color="blue", alpha=0.6, legend_label="map lines")
p.multi_line('crosswalks_xs', 'crosswalks_ys', source = map_crosswalks_source, line_width=1, color="red", alpha = 0.6, legend_label="crosswalks")
p.multi_line('lane_xs', 'lane_ys', source = route_lane_source, line_width=1, color="green", alpha = 0.6, legend_label="route lane")
p.line('xs', 'ys', source = ego_future_gt_source, line_width = 1.5, color = "yellow", alpha = 0.9, legend_label="ego future gt" )

p.multi_line('agents_x', 'agents_y', source = agents_future_gt_source, line_width=1, color="black", alpha = 0.6, legend_label="agents gt traj")
# 创建滑块类，初始化并绑定回调
slider_class = ipywidgets.IntSlider(
    value=0, min=0, max=num_frames-1, step=1, description="Frame"
)

# 滑块回调
def slider_callback(frame):
    print("frame: ", frame)
    new_ego_x = ego_x[frame]
    new_ego_y = ego_y[frame]
    print("new ego_x: ", new_ego_x)
    print("new ego_y: ", new_ego_y)

    # 更新数据
    source.data.update({'ego_x': [new_ego_x], 'ego_y': [new_ego_y]})
    agents_source.data.update( {'agents_x': agents_x[:, frame].tolist(), 'agents_y': agents_y[:, frame].tolist()})

    bkp.show(p, notebook_handle=True)
    push_notebook()


ipywidgets.interactive(slider_callback, frame=slider_class)




Loading BokehJS ...

ego shape: (21, 7)
agents shape: (20, 21, 11)


interactive(children=(IntSlider(value=0, description='Frame', max=20), Output()), _dom_classes=('widget-intera…

In [2]:
# print(map_lanes.shape)
# num_lanes = len(map_lanes)
# num_points_of_lane = len(map_lanes[0])
# print(num_lanes)
# print(num_points_of_lane)

# print(map_crosswalks.shape)
# print(map_crosswalks[4])
# print(route_lanes.shape)
# print(ego_future_gt.shape)

# print(ego_future_gt_xs)
print(agents_future_gt.shape)
print(num_agents)
# print(agents_future_gt[0])
print(ego_future_gt_xs)
print(ego_future_gt_ys)


(10, 80, 3)
20
[0.5736493468284607, 1.1498582363128662, 1.7266513109207153, 2.306126594543457, 2.8809969425201416, 3.459704637527466, 4.035027980804443, 4.610382080078125, 5.181013107299805, 5.755230903625488, 6.330262184143066, 6.906605243682861, 7.487567901611328, 8.072015762329102, 8.657133102416992, 9.245640754699707, 9.83574104309082, 10.426422119140625, 11.018091201782227, 11.601350784301758, 12.185717582702637, 12.769536018371582, 13.350640296936035, 13.930530548095703, 14.510977745056152, 15.09797477722168, 15.677017211914062, 16.255342483520508, 16.826976776123047, 17.40353012084961, 17.97504997253418, 18.53598403930664, 19.09231185913086, 19.649255752563477, 20.195615768432617, 20.739452362060547, 21.283084869384766, 21.815519332885742, 22.348432540893555, 22.8723201751709, 23.38895034790039, 23.893787384033203, 24.397289276123047, 24.88756561279297, 25.36962890625, 25.842910766601562, 26.307058334350586, 26.762720108032227, 27.215545654296875, 27.6569881439209, 28.0881195068